In [1]:
import arcpy
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None


In [2]:
def add_leading_zeroes(parcel_id_str):
    if len(parcel_id_str) == 8:
        return "0{}".format(str(parcel_id_str))
    if len(parcel_id_str) == 7:
        return "00{}".format(str(parcel_id_str))
    else:
        return parcel_id_str

In [3]:
# Load Extended Descriptions
df = pd.read_csv(r".\Inputs\Davis_Extended_Descriptions_January.csv", dtype={'ACCOUNTNO':str, 'BLTASDESCRIPTION':str})

# format account numbers so that they are all 9 characters long
df['ACCOUNTNO'] = df['ACCOUNTNO'].astype(str)
df['ACCOUNTNO'] = df['ACCOUNTNO'].map(add_leading_zeroes)
df['ID_LEN'] = df['ACCOUNTNO'].map(len)

# add field to indicate source
df['source'] = 'xlsx' 
df.head()

,ACCOUNTNO,ACCTTYPE,BLTASDESCRIPTION,APPRAISALTYPE,ID_LEN,source
0,010010001,Vacant Land,NaN,Real,9,xlsx
1,010020001,Vacant Land,NaN,Real,9,xlsx
2,010030001,Vacant Land,NaN,Real,9,xlsx
3,010040001,Vacant Land,NaN,Real,9,xlsx
4,010040002,Vacant Land,NaN,Real,9,xlsx


In [4]:
# ensure all ids are nine characters long
df['ID_LEN'].value_counts()

9    118188
Name: ID_LEN, dtype: int64

In [5]:
# Load Davis County Parcels
davis_parcels = pd.DataFrame.spatial.from_featureclass(r'.\Inputs\Davis_County_LIR_Parcels.gdb\Parcels_Davis_LIR_UTM12')
davis_parcels.head()

,OBJECTID,COUNTY_NAME,COUNTY_ID,ASSESSOR_SRC,BOUNDARY_SRC,DISCLAIMER,CURRENT_ASOF,PARCEL_ID,SERIAL_NUM,PARCEL_ADD,PARCEL_CITY,TAXEXEMPT_TYPE,TAX_DISTRICT,TOTAL_MKT_VALUE,LAND_MKT_VALUE,PARCEL_ACRES,PROP_CLASS,PRIMARY_RES,HOUSE_CNT,SUBDIV_NAME,BLDG_SQFT,BLDG_SQFT_INFO,FLOORS_CNT,FLOORS_INFO,BUILT_YR,EFFBUILT_YR,CONST_MATERIAL,SHAPE
0,1,Davis,6,www.co.davis.ut.us/assessor/,www.co.davis.ut.us/recorder/,http://www.utah.gov/disclaimer.html,2020-07-20,041000016,None,788 E EMERALD HILLS,Bountiful,NO,3,351000.0,205927.0,0.345,Residential,Y,1,None,1362.0,Only above grade areas are counted,1.0,Only above grade floors are counted and only t...,1971.0,None,None,"{""rings"": [[[427114.3192999996, 4524886.955700..."
1,2,Davis,6,www.co.davis.ut.us/assessor/,www.co.davis.ut.us/recorder/,http://www.utah.gov/disclaimer.html,2020-07-20,041000017,None,778 E EMERALD HILLS,Bountiful,NO,3,361000.0,213201.0,0.365,Residential,Y,1,None,1668.0,Only above grade areas are counted,1.0,Only above grade floors are counted and only t...,1971.0,None,None,"{""rings"": [[[427109.0613000002, 4524870.0428],..."
2,3,Davis,6,www.co.davis.ut.us/assessor/,www.co.davis.ut.us/recorder/,http://www.utah.gov/disclaimer.html,2020-07-20,041000018,None,768 E EMERALD HILLS,Bountiful,NO,3,328000.0,202229.0,0.335,Residential,Y,1,None,1451.0,Only above grade areas are counted,1.0,Only above grade floors are counted and only t...,1971.0,None,None,"{""rings"": [[[427092.50370000023, 4524831.50029..."
3,4,Davis,6,www.co.davis.ut.us/assessor/,www.co.davis.ut.us/recorder/,http://www.utah.gov/disclaimer.html,2020-07-20,041000003,None,702 E EMERALD HILLS,Bountiful,NO,3,358000.0,198387.0,0.300,Residential,Y,1,None,1276.0,Only above grade areas are counted,1.0,Only above grade floors are counted and only t...,1971.0,None,None,"{""rings"": [[[426957.61569999997, 4524832.62350..."
4,5,Davis,6,www.co.davis.ut.us/assessor/,www.co.davis.ut.us/recorder/,http://www.utah.gov/disclaimer.html,2020-07-20,041010013,None,1231 S CANYON CREST,Bountiful,NO,3,364000.0,214293.0,0.400,Residential,Y,1,None,2269.0,Only above grade areas are counted,1.0,Only above grade floors are counted and only t...,1962.0,None,None,"{""rings"": [[[427944.4425999997, 4525407.878000..."


In [54]:
# Count number of records for each account no
counts = pd.DataFrame(df['ACCOUNTNO'].value_counts()).reset_index()
counts.columns = ['parcel_id', 'n']
counts.head(40)

,parcel_id,n
0,080880083,39
1,120660081,36
2,060290006,31
3,120210032,27
4,090350047,26
5,091190020,24
6,030210173,22
7,090370060,21
8,120730127,21
9,090210042,21


In [45]:
apt_accounts = df[df['BLTASDESCRIPTION'] == 'Multiple - Residential']['ACCOUNTNO'].tolist()

In [79]:
no_apts = counts[~counts['parcel_id'].isin(apt_accounts)]
# no_apts[no_apts['n'] > 2].to_csv('.\\Inputs\\')
no_apts.head(20)

,parcel_id,n
0,080880083,39
2,060290006,31
3,120210032,27
10,100910119,20
12,010820108,18
15,120650141,16
16,014210002,16
19,090470120,16
22,063500001,15
24,010450082,15


In [87]:
df[df['ACCOUNTNO']=='090370083']

,ACCOUNTNO,ACCTTYPE,BLTASDESCRIPTION,APPRAISALTYPE,ID_LEN,source
52544,090370083,Commercial,Mini Warehouse,Real,9,xlsx
52545,090370083,Commercial,Mini Warehouse,Real,9,xlsx
52546,090370083,Commercial,Mini Warehouse,Real,9,xlsx
52547,090370083,Commercial,Mini Warehouse,Real,9,xlsx
52548,090370083,Commercial,Mini Warehouse,Real,9,xlsx
52549,090370083,Commercial,Mini Warehouse,Real,9,xlsx
52550,090370083,Commercial,Office-Apartment,Real,9,xlsx
52551,090370083,Commercial,Mini Warehouse,Real,9,xlsx
52552,090370083,Commercial,Mini Warehouse,Real,9,xlsx
52553,090370083,Commercial,Mini Warehouse,Real,9,xlsx


In [8]:
davis_parcels_unique = davis_parcels.groupby(['PARCEL_ID']).first().reset_index()
print(davis_parcels.shape)
print(davis_parcels_unique.shape)

(119357, 28)
(114047, 28)


In [9]:
# group by, take first item (not good method)
df_unique = df.groupby(['ACCOUNTNO']).first().reset_index()
print(df.shape)
print(df_unique.shape)

(118188, 6)
(114450, 6)


In [10]:
df_unique = df.groupby('ACCOUNTNO')['BLTASDESCRIPTION'].apply(list).reset_index(name='des_all')

In [17]:
from scipy.stats import mode
mode(['c','c', 'c','a','a', 'a','a','b' 'b', 'b'])

ModeResult(mode=array(['a'], dtype='<U2'), count=array([4]))

In [12]:
def smart_mode(List):
    if len(List) >= 2:
        mode_object = mode(List)
        mode_count = mode_object[2]
        mode_count
        
    
df_unique['des_mode'] = df_unique['des_all'].apply(mode)
df_unique.to_csv('des_by_id.csv')

In [13]:
# count number of matching records
davis_parcels_unique.merge(df_unique, left_on='PARCEL_ID', right_on='ACCOUNTNO', how='inner')

,PARCEL_ID,OBJECTID,COUNTY_NAME,COUNTY_ID,ASSESSOR_SRC,BOUNDARY_SRC,DISCLAIMER,CURRENT_ASOF,SERIAL_NUM,PARCEL_ADD,PARCEL_CITY,TAXEXEMPT_TYPE,TAX_DISTRICT,TOTAL_MKT_VALUE,LAND_MKT_VALUE,PARCEL_ACRES,PROP_CLASS,PRIMARY_RES,HOUSE_CNT,SUBDIV_NAME,BLDG_SQFT,BLDG_SQFT_INFO,FLOORS_CNT,FLOORS_INFO,BUILT_YR,EFFBUILT_YR,CONST_MATERIAL,SHAPE,ACCOUNTNO,des_all,des_mode
0,010010001,1128,Davis,6,www.co.davis.ut.us/assessor/,www.co.davis.ut.us/recorder/,http://www.utah.gov/disclaimer.html,2020-07-20,None,,None,YES,88,2631555.0,2631555.0,584.790,Vacant Land,N,None,None,0.0,Only above grade areas are counted,0.0,Only above grade floors are counted and only t...,NaN,None,None,"{'rings': [[[432837.2489, 4523084.3905], [4344...",010010001,[nan],"([nan], [1])"
1,010020001,745,Davis,6,www.co.davis.ut.us/assessor/,www.co.davis.ut.us/recorder/,http://www.utah.gov/disclaimer.html,2020-07-20,None,,None,YES,88,2862855.0,2862855.0,636.190,Vacant Land,N,None,None,0.0,Only above grade areas are counted,0.0,Only above grade floors are counted and only t...,NaN,None,None,"{'rings': [[[431259.5963000003, 4523102.8245],...",010020001,[nan],"([nan], [1])"
2,010030001,84167,Davis,6,www.co.davis.ut.us/assessor/,www.co.davis.ut.us/recorder/,http://www.utah.gov/disclaimer.html,2020-07-20,None,,None,YES,88,2820735.0,2820735.0,626.830,Vacant Land,N,None,None,0.0,Only above grade areas are counted,0.0,Only above grade floors are counted and only t...,NaN,None,None,"{'rings': [[[431298.3334999997, 4521492.8399],...",010030001,[nan],"([nan], [1])"
3,010040001,84181,Davis,6,www.co.davis.ut.us/assessor/,www.co.davis.ut.us/recorder/,http://www.utah.gov/disclaimer.html,2020-07-20,None,,None,YES,1,720000.0,720000.0,160.000,Vacant Land,N,None,None,0.0,Only above grade areas are counted,0.0,Only above grade floors are counted and only t...,NaN,None,None,"{'rings': [[[429278.03220000025, 4523132.31059...",010040001,[nan],"([nan], [1])"
4,010040002,4596,Davis,6,www.co.davis.ut.us/assessor/,www.co.davis.ut.us/recorder/,http://www.utah.gov/disclaimer.html,2020-07-20,None,,Bountiful,NO,1,627025.0,627025.0,71.420,Vacant Land,N,None,None,0.0,Only above grade areas are counted,0.0,Only above grade floors are counted and only t...,NaN,None,None,"{'rings': [[[428065.1196999997, 4522371.7095],...",010040002,[nan],"([nan], [1])"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114032,550030001,84489,Davis,6,www.co.davis.ut.us/assessor/,www.co.davis.ut.us/recorder/,http://www.utah.gov/disclaimer.html,2020-07-20,None,302 E 650 NORTH,Clearfield,NO,14,214922.0,58600.0,0.223,Commercial,N,None,None,1384.0,Only above grade areas are counted,0.0,Only above grade floors are counted and only t...,1950.0,None,None,"{'rings': [[[414211.6102, 4553028.373], [41422...",550030001,[Office Building],"([Office Building], [1])"
114033,550030002,105600,Davis,6,www.co.davis.ut.us/assessor/,www.co.davis.ut.us/recorder/,http://www.utah.gov/disclaimer.html,2020-07-20,None,306 E 650 NORTH,Clearfield,NO,14,2797906.0,305521.0,1.460,Commercial,N,1,None,9886.0,Only above grade areas are counted,1.0,Only above grade floors are counted and only t...,2014.0,None,None,"{'rings': [[[414302.11479999963, 4552902.90530...",550030002,[Neighborhood Shopping Center],"([Neighborhood Shopping Center], [1])"
114034,550040001,105601,Davis,6,www.co.davis.ut.us/assessor/,www.co.davis.ut.us/recorder/,http://www.utah.gov/disclaimer.html,2020-07-20,None,,Clearfield,NO,14,2848745.0,271413.0,1.276,Commercial,N,1,None,20968.0,Only above grade areas are counted,1.0,Only above grade floors are counted and only t...,2017.0,None,None,"{'rings': [[[414561.9134999998, 4552696.2083],...",550040001,[Distribution Warehouse],"([Distribution Warehouse], [1])"
114035,550050005,101889,Davis,6,www.co.davis.ut.us/assessor/,www.co.davis.ut.us/recorder/,http://www.utah.gov/disclaimer.html,2020-07-20,None,,Clearfield,NO,14,20723538.0,981752.0,5.511,Commercial,N,None,None,76778.0,O

In [14]:
# Count number of records for each parcel
parcel_id_counts = pd.DataFrame(davis_parcels['PARCEL_ID'].value_counts()).reset_index()
parcel_id_counts.columns = ['PARCEL_ID', 'id_count']
parcels_with_counts = davis_parcels.merge(parcel_id_counts, left_on='PARCEL_ID', right_on='PARCEL_ID', how='left')


# export parcels to examine which ones had information added to them from the excel sheet
parcels_export = parcels_with_counts.merge(df_unique, left_on='PARCEL_ID', right_on='ACCOUNTNO', how='left')
parcels_export = parcels_export[['PARCEL_ID','BUILT_YR','id_count', 'source','SHAPE']].copy()
# parcels_export.spatial.to_featureclass(location=r'E:\Projects\REMM-Input-Data-Prep-2019\Parcels\2020-Davis\Inputs\Davis_County_LIR_Parcels.gdb\parcels_with_extended_attrs_jan')

KeyError: "['source'] not in index"

### SUMMARY

Version 1:
- 114047 unique parcels in the Davis County lir parcel dataset
- 116386 unique account numbers in extended descriptions excel file
- 97% (113215 of the 116386) unique account numbers are present in the lir parcel database (3,171 did not join)

Version 2 (January data):
- 114047 unique parcels in the Davis County lir parcel dataset
- 114450 unique account numbers in extended descriptions excel file
- 98% (114037 of the 116386) unique account numbers are present in the lir parcel database (2,349 did not join)